In [41]:
import pandas as pd 
df1=pd.read_csv('mtcars2.csv',encoding='cp949')
#print(df1)
mpg=df1.mpg.sort_values(ascending=False)
mpg_10=mpg.iloc[9]
idx=mpg[:10].index
df1.loc[idx,'mpg']=mpg_10
print(round(df1[df1['drat']>=4]['mpg'].mean(),3))

20.671


In [48]:
df2=pd.read_csv('mtcars2.csv',encoding='cp949')
l=len(df2)
train=df2[:int(l*0.8)]
changed=train['disp'].fillna(train['disp'].median())
yet=train['disp'].std()
later=changed.std()
print(abs(round(later-yet,3)))

7.966


In [50]:
df3=pd.read_csv('gehan.csv',encoding='cp949')
time=df3['time']
time_mean=time.mean()
time_std=time.std()
time[(time<time_mean-1.5*time_std)|(time>time_mean+1.5*time_std)].sum()

133

In [66]:
X_train=pd.read_csv('stroke_X_train.csv',encoding='cp949')
X_test=pd.read_csv('stroke_X_test.csv',encoding='cp949')
y_train=pd.read_csv('stroke_y_train.csv',encoding='cp949')

id=X_test.id

y_train.drop(columns=['id'],inplace=True)
#print(X_train.head())
obj=['gender','hypertension','heart_disease','ever_married','work_type','Residence_type','smoking_status']
nums=['age','bmi']

# id 삭제
X_train.drop(columns=['id'],inplace=True)
X_test.drop(columns=['id'],inplace=True)

from sklearn.model_selection import train_test_split
X_train,X_val,y_train,y_val=train_test_split(X_train,y_train,test_size=0.2,random_state=1069)

from sklearn.preprocessing import OneHotEncoder
encoder= OneHotEncoder(sparse=False)
encoder.fit(X_train[obj])
X_train_oh=encoder.transform(X_train[obj])
X_val_oh=encoder.transform(X_val[obj])
X_test_oh=encoder.transform(X_test[obj])

from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaler.fit(X_train[nums])
X_train_sc=scaler.transform(X_train[nums])
X_val_sc=scaler.transform(X_val[nums])
X_test_sc=scaler.transform(X_test[nums])

import numpy as np
X_Train=np.concatenate([X_train_oh,X_train_sc],axis=1)
X_Val=np.concatenate([X_val_oh,X_val_sc],axis=1)
X_Test=np.concatenate([X_test_oh,X_test_sc],axis=1)

from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.metrics import accuracy_score

# rt
rf_model=RandomForestClassifier(n_estimators=300)
rf_model.fit(X_Train,y_train)
rf_pred=rf_model.predict(X_Val)
rf_score=accuracy_score(y_val,rf_pred)
print('rf',rf_score)

# # ada
# ada_model=AdaBoostClassifier()
# ada_model.fit(X_Train,y_train)
# ada_pred=ada_model.predict(X_Val)
# ada_score=accuracy_score(y_val,ada_pred)
# print('ada',ada_score)

result=rf_model.predict(X_Test)
data=pd.DataFrame({'id':id,'stroke':result})
data.to_csv('result.csv',encoding='cp949',index=False)

c:\Users\KDP-30\anaconda3\envs\ML_38\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
c:\Users\KDP-30\anaconda3\envs\ML_38\lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


rf 0.94375


In [68]:
pre=pd.read_csv('result.csv',encoding='cp949')
cor=pd.read_csv('stroke_y_test.csv',encoding='cp949')
accuracy_score(cor['stroke'],pre['stroke'])

0.938788257339163